In [1]:
import pandas as pd
import numpy as np
from os import listdir

In [2]:
df1 = pd.read_csv("./csv/events_up_to_01062018.csv",low_memory=False)

In [3]:
df2 = pd.read_csv("./csv/labels_training_set.csv")

In [4]:
df1['timestamp'] = pd.to_datetime(df1['timestamp'])

In [5]:
def modelo(obj):
    if(pd.isnull(obj)):
        return obj
    
    list = obj.split(" ")
    if len(list) == 1:
        return obj
    if "iPhone" in list:
        return "iPhone"
    if "Samsung" in list:
        return "Samsung"
    if "Motorola" in list:
        return "Motorola"
    if "Lenovo" in list:
        return "Lenovo"
    if "Sony" in list:
        return "Sony"
    if "LG" in list:
        return "LG"
    if "Asus" in list:
        return "Asus"
    if "iPad" in list:
        return "iPad"
    if "Quantum" in list:
        return "Quantum"
    return obj

df1['model'] = df1['model'].apply(modelo)

In [6]:
df1['model'].value_counts()

iPhone              593603
Samsung             501307
Motorola            162914
LG                   24814
Sony                 15599
Lenovo               11382
iPad                  5663
Asus                  4536
Quantum               1693
Outros TV LED 15         1
Xiaomi Redmi 2           1
Name: model, dtype: int64

In [7]:
df1['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [8]:
df1['storage'].value_counts()

16GB     442096
32GB     430283
64GB     228413
128GB     99345
8GB       96019
256GB     17623
4GB        5409
512MB      1342
Name: storage, dtype: int64

In [9]:
def convertir_storage(obj):
    number = ""
    if(pd.isnull(obj)):
        return obj
    for char in obj:
        if (char != 'G') & (char != 'B') & (char != 'M'):
            number += char
    return int(number)

In [10]:
df1['storage'] = df1['storage'].apply(convertir_storage)

In [11]:
df1['storage'].value_counts()

16.0     442096
32.0     430283
64.0     228413
128.0     99345
8.0       96019
256.0     17623
4.0        5409
512.0      1342
Name: storage, dtype: int64

# Aalisis para evento convercion

In [12]:
df1 = df1.loc[df1['timestamp'].dt.month == 5,:]

In [13]:
df1.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung,Excelente,32.0,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone,Muito Bom,32.0,Cinza espacial,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone,Muito Bom,64.0,Prateado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone,Bom,128.0,Vermelho,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG,Excelente,32.0,Branco,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
df_a = df1.loc[df1['event']=='conversion',:]

In [15]:
df_a.isnull().sum()

timestamp                      0
event                          0
person                         0
url                         5498
sku                            0
model                          0
condition                      0
storage                        0
color                          0
skus                        5388
search_term                 5498
staticpage                  5498
campaign_source             5498
search_engine               5498
channel                     5498
new_vs_returning            5498
city                        5498
region                      5498
country                     5498
device_type                 5498
screen_resolution           5498
operating_system_version    5498
browser_version             5498
dtype: int64

In [16]:
df_a_model = df_a.loc[:,['person','model','storage','color']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model"})
        
df_a_storage = df_a.loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage"})
        
df_a_condition = df_a.loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition"})
        
df_a_color = df_a.loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color"})

In [17]:
df_datos_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\

In [18]:
df_datos_convercion.to_csv("df_datos_convercion.csv", index = True, header = True, sep = ',', 
          encoding = 'utf-8-sig')

# Analisis para los eventos distintos de convercion

In [19]:
df_a = df1.loc[df1['event']!='conversion',:]

In [20]:
df_a_model = df_a.dropna(subset=['model']).loc[:,['person','model']].groupby('person')\
                .apply(lambda x: x['model'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"model_no_convercion"})


df_a_storage = df_a.dropna(subset=['storage']).loc[:,['person','storage']].groupby('person')\
                .apply(lambda x: x['storage'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"storage_no_convercion"})
        
        
df_a_condition = df_a.dropna(subset=['condition']).loc[:,['person','condition']].groupby('person')\
                .apply(lambda x: x['condition'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"condition_no_convercion"})
        
        
df_a_color = df_a.dropna(subset=['color']).loc[:,['person','color']].groupby('person')\
                .apply(lambda x: x['color'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"color_no_convercion"})

In [21]:
df_datos_no_convercion = df_a_model.merge(df_a_storage,how='left',on='person')\
                                .merge(df_a_condition,how='left',on='person')\
                                .merge(df_a_color,how='left',on='person')\

In [22]:
df_datos_no_convercion.to_csv("df_datos_no_convercion.csv", index = True, header = True, sep = ',', 
          encoding = 'utf-8-sig')

# Mergeo de los DF y procesamiento

In [23]:
DF_a_procesar = df_datos_convercion.merge(df_datos_no_convercion,how='left',on='person')\
                                    .merge(df2,how='left',on='person')

In [24]:
DF_a_procesar = DF_a_procesar.dropna(subset=['label'])

In [25]:
def procesar_df(df):
    df_2 = df
    for element in df.index.tolist():
        label = df.at[element,'label']
        if label==0:
            person = df.at[element,'person']
            df_2.loc[df_2.person==person,['model']] = df.at[element,'model_no_convercion']
            df_2.loc[df_2.person==person,['storage']] = df.at[element,'storage_no_convercion']
            df_2.loc[df_2.person==person,['condition']] = df.at[element,'condition_no_convercion']
            df_2.loc[df_2.person==person,['color']] = df.at[element,'color_no_convercion']
    df_2 = df_2.loc[:,['person','model','storage','condition','color','label']]
    return df_2

In [26]:
dffinal=procesar_df(DF_a_procesar)

Le agrego el serch_engine a la persona

In [27]:
dfa = df1

In [28]:
df_a_search_engine = dfa.dropna(subset=['search_engine']).loc[:,['person','search_engine']].groupby('person')\
                .apply(lambda x: x['search_engine'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"search_engine"})
df_a_search_engine.head()

,person,search_engine
0,000ba417,Google
1,000c79fe,Google
2,000e4d9e,Google
3,000e619d,Google
4,0019e639,Google


In [29]:
dffinal = dffinal.merge(df_a_search_engine,how='left',on='person')

In [30]:
dffinal = dffinal.dropna(subset=['search_engine'])
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1395 entries, 0 to 1872
Data columns (total 7 columns):
person           1395 non-null object
model            1395 non-null object
storage          1395 non-null float64
condition        1395 non-null object
color            1395 non-null object
label            1395 non-null float64
search_engine    1395 non-null object
dtypes: float64(2), object(5)
memory usage: 87.2+ KB


In [31]:
BRS = ["Vivaldi","Chrome","Android","Safari","Chromium","Opera","Firefox","UC","Facebook","Edge","Samsung","IE"]
i = 0
def brow(obj) :
    if(pd.isnull(obj)):
        return obj
    if(obj == ""):
        return float("NaN")
    list = obj.split(" ")
    true_list = []
    for word in list:
        if word in BRS:
            true_list.append(word)
    if( len(true_list) == 0):
        #print (' '.join(word for word in list))
        return "Other"
    
    return ' '.join(word for word in true_list)
    
df1["browser"] = df1["browser_version"].apply(brow)

In [37]:
dfb = df1
df_a_browser = dfb.dropna(subset=['browser']).loc[:,['person','browser']].groupby('person')\
                .apply(lambda x: x['browser'].value_counts().idxmax())\
                .reset_index().rename(columns={0:"browser"})
df_a_browser.head()

,person,browser
0,0008ed71,Chrome
1,00091926,Chrome
2,000ba417,Chrome
3,000c79fe,Chrome
4,000e4d9e,Chrome


In [35]:
dffinal = dffinal.merge(df_a_browser,how='left',on='person')

In [36]:
dffinal = dffinal.dropna(subset=['search_engine'])
dffinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1395 entries, 0 to 1394
Data columns (total 8 columns):
person           1395 non-null object
model            1395 non-null object
storage          1395 non-null float64
condition        1395 non-null object
color            1395 non-null object
label            1395 non-null float64
search_engine    1395 non-null object
browser          1392 non-null object
dtypes: float64(2), object(6)
memory usage: 98.1+ KB


Escribo el data frame para usar machine learning

In [39]:
dffinal.to_csv("df_a_procesar.csv", index = True, header = True, sep = ',', 
          encoding = 'utf-8-sig')